In [1]:
import pandas as pd
import numpy as np
import psycopg2
import getpass
import gc
from tqdm import tqdm
from collections import Counter

In [2]:
user = 'postgres'
host = '10.10.116.166'
#127.0.0.1
#host = '127.0.0.1'
port = 8088
dbname = 'mimiciv'
schema = 'public, mimiciv_derived, mimiciv_hosp, mimiciv_icu'

In [4]:
con = psycopg2.connect(user=user, host=host, port=port,
                      dbname=dbname, password=getpass.getpass())
cur = con.cursor()

········


In [39]:
DATA_PATH = './data_rsmp6.tsv'
RESAMPLE_INTERVAL = '6H'
OUTPUT_FILENAME = './data.tsv'

# add ventilation, sedatives and vasopressors

In [40]:
data = pd.read_csv(DATA_PATH, sep='\t')

In [41]:
data.head()

,stay_id,charttime,heartrate,sysbp,diasbp,meanbp,resprate,tempc,spo2,aniongap,...,pt,sodium,bun,wbc,glucose,creatinine,uo_rt_6hr,uo_rt_12hr,uo_rt_24hr,aki_stage
0,30000646,2194-04-29 00:00:00,94.666667,102.200000,62.400000,71.400000,25.666667,37.111111,96.000000,12.0,...,14.5,138.0,12.0,8.5,102.0,0.9,NaN,NaN,NaN,0.0
1,30000646,2194-04-29 06:00:00,89.880000,92.227273,51.380952,59.952381,29.826087,37.722222,96.375000,12.0,...,14.3,141.0,12.0,10.6,108.0,1.0,NaN,NaN,NaN,0.0
2,30000646,2194-04-29 12:00:00,72.500000,94.500000,60.333333,68.166667,18.833333,36.833333,98.666667,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,1.6727,NaN,NaN,0.0
3,30000646,2194-04-29 18:00:00,83.888889,91.235294,56.294118,64.000000,22.944444,37.000000,94.500000,13.0,...,NaN,143.0,9.0,7.9,144.0,0.6,2.4921,2.5419,NaN,0.0
4,30000646,2194-04-30 00:00:00,79.125000,84.125000,52.875000,60.000000,16.250000,36.583333,96.000000,10.0,...,16.8,140.0,7.0,7.2,86.0,NaN,NaN,NaN,NaN,NaN


In [42]:
data.charttime = pd.to_datetime(data.charttime)

In [43]:
icustays = data.stay_id.unique().tolist()
len(icustays)

25691

In [45]:
# 数据库查询语句
ventilation = 'select * from mimiciv_derived.ventilation'
sedative = 'select * from sedative'
vasopressor = 'select * from mimiciv_derived.vasopressin'
rrt = 'select * from rrt_all_admission_0721'
crrt = 'select * from mimiciv_derived.crrt'

query_dict = {
    'ventilation': ventilation,
    'sedative' : sedative
    'vasopressor': vasopressor,
    'rrt': rrt,
    'crrt': crrt,
}

In [46]:
for name, query in tqdm(query_dict.items(), ncols=100):
    table = pd.read_sql_query(query, con)
    table = table[table.stay_id.isin(icustays)]
    if pd.isna(table).any().any():
        print('{0} has None!'.format(name))
        continue
    table = table[['stay_id', 'starttime', 'endtime']]
    table['starttime'] = pd.to_datetime(table['starttime'])
    table['endtime'] = pd.to_datetime(table['endtime'])
    table.drop_duplicates(inplace=True)
    table.sort_values(by=['stay_id', 'starttime'], inplace=True, ascending=True, ignore_index=True)
    
    table_group = table.groupby('stay_id')
    data_group = data.copy()
    data_group[name] = 0
    data_group = data_group.set_index('charttime').groupby('stay_id')
    
    result_list = []
    for i in icustays:
        tmp_data = data_group.get_group(i).copy()
        try:
            tmp_table = table_group.get_group(i).copy()
            for idx in tmp_table.index:
                starttime, endtime = tmp_table.loc[idx, 'starttime'], tmp_table.loc[idx, 'endtime']
                tmp_data.loc[starttime:endtime, name] = 1
        except:
            pass
        result_list.append(tmp_data.reset_index(drop=False))
    data = pd.concat(result_list, axis=0, ignore_index=True)
#     print('{0} first: {1}'.format(name, Counter(data[name])))
    
    data_group = data.groupby('stay_id')
    table_start = table[['stay_id', 'starttime']]
    table_end = table[['stay_id', 'endtime']]
    table_end.columns = ['stay_id', 'starttime']
    table = pd.concat([table_start, table_end], axis=0)
    table.sort_values(by=['stay_id', 'starttime'], axis=0, ascending=True, inplace=True)
    table.drop_duplicates(inplace=True)
    table_group = table.set_index('starttime').groupby('stay_id')
    
    result_list = []
    for i in icustays:
        tmp_data = data_group.get_group(i).copy()
        try:
            tmp_table = table_group.get_group(i).copy()
            for idx in tmp_data.index:
                starttime = tmp_data.loc[idx, 'charttime']
                endtime = starttime + pd.Timedelta(RESAMPLE_INTERVAL) - pd.Timedelta('1s')
                if len(tmp_table.loc[starttime:endtime, ]) == 0:
                    pass
                else:
                    tmp_data.loc[idx, name] = 1
        except:
            pass
        result_list.append(tmp_data)
    data = pd.concat(result_list, axis=0, ignore_index=True)
#     print('{0} second: {1}'.format(name, Counter(data[name])))
    
    del table, table_start, table_end, table_group, tmp_data, tmp_table, data_group, result_list
    gc.collect()

100%|████████████████████████████████████████████████████████████████| 4/4 [09:31<00:00, 142.93s/it]

crrt has None!


In [47]:
data.columns

Index(['charttime', 'stay_id', 'heartrate', 'sysbp', 'diasbp', 'meanbp',
       'resprate', 'tempc', 'spo2', 'aniongap', 'bicarbonate', 'chloride',
       'hematocrit', 'hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt',
       'sodium', 'bun', 'wbc', 'glucose', 'creatinine', 'uo_rt_6hr',
       'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage', 'ventilation', 'vasopressor',
       'rrt'],
      dtype='object')

In [48]:
# 数据库查询语句
#ventilation = 'select * from ventilation'
ventilation = 'select * from mimiciv_derived.ventilation'
sedative = 'select * from sedative'
vasopressor = 'select * from mimiciv_derived.vasopressin'
rrt = 'select * from rrt_all_admission_0721'
crrt = 'select * from mimiciv_derived.crrt'
#adenosine = 'select * from adenosine'
#isuprel = 'select * from isuprel'
query_dict = {
    #'ventilation': ventilation,
    'sedative': sedative,
    #'vasopressor': vasopressor,
    #'rrt': rrt,
    #'crrt': crrt,
    #'adenosine': adenosine,
    #'isuprel': isuprel
}
for name, query in tqdm(query_dict.items(), ncols=100):
    table = pd.read_sql_query(query, con)
    table = table[table.stay_id.isin(icustays)]
    if pd.isna(table).any().any():
        print('{0} has None!'.format(name))
        continue
    table = table[['stay_id', 'starttime', 'endtime']]
    table['starttime'] = pd.to_datetime(table['starttime'])
    table['endtime'] = pd.to_datetime(table['endtime'])
    table.drop_duplicates(inplace=True)
    table.sort_values(by=['stay_id', 'starttime'], inplace=True, ascending=True, ignore_index=True)
    
    table_group = table.groupby('stay_id')
    data_group = data.copy()
    data_group[name] = 0
    data_group = data_group.set_index('charttime').groupby('stay_id')
    
    result_list = []
    for i in icustays:
        tmp_data = data_group.get_group(i).copy()
        try:
            tmp_table = table_group.get_group(i).copy()
            for idx in tmp_table.index:
                starttime, endtime = tmp_table.loc[idx, 'starttime'], tmp_table.loc[idx, 'endtime']
                tmp_data.loc[starttime:endtime, name] = 1
        except:
            pass
        result_list.append(tmp_data.reset_index(drop=False))
    data = pd.concat(result_list, axis=0, ignore_index=True)
#     print('{0} first: {1}'.format(name, Counter(data[name])))
    
    data_group = data.groupby('stay_id')
    table_start = table[['stay_id', 'starttime']]
    table_end = table[['stay_id', 'endtime']]
    table_end.columns = ['stay_id', 'starttime']
    table = pd.concat([table_start, table_end], axis=0)
    table.sort_values(by=['stay_id', 'starttime'], axis=0, ascending=True, inplace=True)
    table.drop_duplicates(inplace=True)
    table_group = table.set_index('starttime').groupby('stay_id')
    
    result_list = []
    for i in icustays:
        tmp_data = data_group.get_group(i).copy()
        try:
            tmp_table = table_group.get_group(i).copy()
            for idx in tmp_data.index:
                starttime = tmp_data.loc[idx, 'charttime']
                endtime = starttime + pd.Timedelta(RESAMPLE_INTERVAL) - pd.Timedelta('1s')
                if len(tmp_table.loc[starttime:endtime, ]) == 0:
                    pass
                else:
                    tmp_data.loc[idx, name] = 1
        except:
            pass
        result_list.append(tmp_data)
    data = pd.concat(result_list, axis=0, ignore_index=True)
#     print('{0} second: {1}'.format(name, Counter(data[name])))
    
    del table, table_start, table_end, table_group, tmp_data, tmp_table, data_group, result_list
    gc.collect()

100%|████████████████████████████████████████████████████████████████| 1/1 [07:14<00:00, 434.63s/it]


In [49]:
data.columns

Index(['charttime', 'stay_id', 'heartrate', 'sysbp', 'diasbp', 'meanbp',
       'resprate', 'tempc', 'spo2', 'aniongap', 'bicarbonate', 'chloride',
       'hematocrit', 'hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt',
       'sodium', 'bun', 'wbc', 'glucose', 'creatinine', 'uo_rt_6hr',
       'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage', 'ventilation', 'vasopressor',
       'rrt', 'sedative'],
      dtype='object')

# add time independent data

In [52]:
details = 'select * from icustay_detail_0721'
details = pd.read_sql_query(details, con)

In [53]:
details.head()

,subject_id,hadm_id,stay_id,gender,admittime,dischtime,admission_age,race,admission_type
0,10001884,26184834,37510196,F,2131-01-07 20:39:00,2131-01-20 05:15:00,77.018296,BLACK/AFRICAN AMERICAN,OBSERVATION ADMIT
1,10002155,23822395,33685454,F,2129-08-04 12:44:00,2129-08-18 16:53:00,81.592179,WHITE,EW EMER.
2,10002155,28994087,31090461,F,2130-09-23 21:59:00,2130-09-29 18:55:00,82.729467,WHITE,EW EMER.
3,10002348,22725460,32610785,F,2112-11-30 22:22:00,2112-12-10 17:56:00,77.917014,WHITE,OBSERVATION ADMIT
4,10002428,20321825,34807493,F,2156-04-30 20:35:00,2156-05-03 16:36:00,81.330235,WHITE,EW EMER.


In [54]:
details.drop(['hadm_id', 'subject_id', 'admittime', 'dischtime'], axis=1, inplace=True)
icustays = data.stay_id.unique().tolist()

details = details[details.stay_id.isin(icustays)]
details.head()

,stay_id,gender,admission_age,race,admission_type
0,37510196,F,77.018296,BLACK/AFRICAN AMERICAN,OBSERVATION ADMIT
1,33685454,F,81.592179,WHITE,EW EMER.
2,31090461,F,82.729467,WHITE,EW EMER.
3,32610785,F,77.917014,WHITE,OBSERVATION ADMIT
5,35479615,F,81.359694,WHITE,EW EMER.


In [55]:
details = details[details.stay_id.isin(icustays)]
details.head()

,stay_id,gender,admission_age,race,admission_type
0,37510196,F,77.018296,BLACK/AFRICAN AMERICAN,OBSERVATION ADMIT
1,33685454,F,81.592179,WHITE,EW EMER.
2,31090461,F,82.729467,WHITE,EW EMER.
3,32610785,F,77.917014,WHITE,OBSERVATION ADMIT
5,35479615,F,81.359694,WHITE,EW EMER.


In [56]:
details.dtypes

stay_id             int64
gender             object
admission_age     float64
race               object
admission_type     object
dtype: object

In [57]:
details.stay_id.nunique()

25691

In [58]:
details.gender.nunique(), details.race.nunique(), details.admission_type.nunique()

(2, 33, 8)

In [59]:
details.race.replace({'ASIAN - ASIAN INDIAN': 'ASIAN', 'ASIAN - CHINESE': 'ASIAN',
                      'ASIAN - KOREAN': 'ASIAN', 'ASIAN - SOUTH EAST ASIAN': 'ASIAN',
                      'BLACK/AFRICAN AMERICAN': 'BLACK', 'BLACK/CARIBBEAN ISLAND': 'BLACK',
                      'BLACK/AFRICAN': 'BLACK', 'BLACK/AFRICAN AMERICAN': 'BLACK',
                      'HISPANIC OR LATINO': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - CENTRAL AMERICAN': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - COLUMBIAN': 'HISPANIC/LATINO', 'HISPANIC/LATINO - CUBAN': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - DOMINICAN': 'HISPANIC/LATINO', 'HISPANIC/LATINO - GUATEMALAN': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - HONDURAN': 'HISPANIC/LATINO', 'HISPANIC/LATINO - MEXICAN': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - PUERTO RICAN': 'HISPANIC/LATINO',
                      'HISPANIC/LATINO - SALVADORAN': 'HISPANIC/LATINO',
                      'WHITE - BRAZILIAN': 'WHITE', 'WHITE - EASTERN EUROPEAN': 'WHITE',
                      'WHITE - OTHER EUROPEAN': 'WHITE', 'WHITE - RUSSIAN': 'WHITE',
                      'UNABLE TO OBTAIN': 'UNKNOWN', 'PATIENT DECLINED TO ANSWER': 'UNKNOWN',
                      'OTHER': 'UNKNOWN','BLACK/CAPE VERDEAN': 'BLACK'
                     }, inplace=True)

In [60]:
details.gender.nunique(), details.race.nunique(), details.admission_type.nunique()

(2, 10, 8)

In [61]:
Counter(details.gender.values)

Counter({'F': 11130, 'M': 14561})

In [62]:
details.admission_age.describe()

count    25691.000000
mean        64.403473
std         15.663131
min         18.002527
25%         55.158227
50%         66.597893
75%         76.585708
max         88.992883
Name: admission_age, dtype: float64

In [63]:
details = pd.get_dummies(details).astype('float32')

In [64]:
details.shape

(25691, 22)

In [65]:
details.head()

,stay_id,admission_age,gender_F,gender_M,race_AMERICAN INDIAN/ALASKA NATIVE,race_ASIAN,race_BLACK,race_HISPANIC/LATINO,race_MULTIPLE RACE/ETHNICITY,race_NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER,...,race_UNKNOWN,race_WHITE,admission_type_DIRECT EMER.,admission_type_DIRECT OBSERVATION,admission_type_ELECTIVE,admission_type_EU OBSERVATION,admission_type_EW EMER.,admission_type_OBSERVATION ADMIT,admission_type_SURGICAL SAME DAY ADMISSION,admission_type_URGENT
0,37510196.0,77.018295,1.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
1,33685456.0,81.592178,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
2,31090460.0,82.729469,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0
3,32610784.0,77.917015,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0
5,35479616.0,81.359695,1.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,1.0,0.0,0.0,0.0,0.0,1.0,0.0,0.0,0.0


In [66]:
data = pd.merge(data, details, how='left', on='stay_id')

In [67]:
data.columns

Index(['charttime', 'stay_id', 'heartrate', 'sysbp', 'diasbp', 'meanbp',
       'resprate', 'tempc', 'spo2', 'aniongap', 'bicarbonate', 'chloride',
       'hematocrit', 'hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt',
       'sodium', 'bun', 'wbc', 'glucose', 'creatinine', 'uo_rt_6hr',
       'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage', 'ventilation', 'vasopressor',
       'rrt', 'sedative', 'admission_age', 'gender_F', 'gender_M',
       'race_AMERICAN INDIAN/ALASKA NATIVE', 'race_ASIAN', 'race_BLACK',
       'race_HISPANIC/LATINO', 'race_MULTIPLE RACE/ETHNICITY',
       'race_NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'race_PORTUGUESE',
       'race_SOUTH AMERICAN', 'race_UNKNOWN', 'race_WHITE',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EU OBSERVATION',
       'admission_type_EW EMER.', 'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGEN

In [48]:
data.columns

Index(['charttime', 'stay_id', 'heartrate', 'sysbp', 'diasbp', 'meanbp',
       'resprate', 'tempc', 'spo2', 'aniongap', 'bicarbonate', 'chloride',
       'hematocrit', 'hemoglobin', 'platelet', 'potassium', 'ptt', 'inr', 'pt',
       'sodium', 'bun', 'wbc', 'glucose', 'creatinine', 'uo_rt_6hr',
       'uo_rt_12hr', 'uo_rt_24hr', 'aki_stage', 'ventilation', 'vasopressor',
       'rrt', 'sedative', 'admission_age', 'gender_F', 'gender_M',
       'race_AMERICAN INDIAN/ALASKA NATIVE', 'race_ASIAN', 'race_BLACK',
       'race_HISPANIC/LATINO', 'race_MULTIPLE RACE/ETHNICITY',
       'race_NATIVE HAWAIIAN OR OTHER PACIFIC ISLANDER', 'race_PORTUGUESE',
       'race_SOUTH AMERICAN', 'race_UNKNOWN', 'race_WHITE',
       'admission_type_DIRECT EMER.', 'admission_type_DIRECT OBSERVATION',
       'admission_type_ELECTIVE', 'admission_type_EW EMER.',
       'admission_type_OBSERVATION ADMIT',
       'admission_type_SURGICAL SAME DAY ADMISSION', 'admission_type_URGENT'],
      dtype='object')

In [70]:
aki = data.aki_stage.unique().tolist()
len(aki)

4

In [75]:
data.to_csv(OUTPUT_FILENAME, sep='\t', index=False)

In [76]:
cur.close()
con.close()